# ETL of secondary datasets

# "Significant Earthqueake Database"


In [18]:
import pandas as pd
import numpy as np
import json 
import csv


In [19]:
db = pd.read_csv('Datasets\Input\significant-earthquake-database (1).csv', sep=';')
db

ID Earthquake Flag Tsunami  Year  Month   Day  Focal Depth  EQ Primary  \
0              4465      Tsunami  1968    9.0  25.0        114.0         5.7   
1              4471          NaN  1969    1.0   3.0         11.0         5.6   
2              4493          NaN  1969    8.0  11.0         33.0         4.7   
3              8089          NaN  1969    9.0  14.0          NaN         5.5   
4              4546          NaN  1970    9.0  26.0          8.0         6.6   
...             ...          ...   ...    ...   ...          ...         ...   
6203          10370          NaN  2018   10.0   7.0         12.0         5.9   
6204          10374          NaN  2018   11.0  14.0          9.0         5.6   
6205          10409          NaN  2019    5.0   6.0        127.0         7.2   
6206          10457      Tsunami  2019   11.0  14.0         33.0         7.1   
6207          10460          NaN  2019   11.0  25.0         10.0         5.0   

      Mw Magnitude  Ms Magnitude  Mb Magnitude  ...  \
0              NaN           NaN           NaN  ...   
1              NaN           NaN           NaN  ...   
2              NaN           4.7           4.6  ...   
3              NaN           5.5           NaN  ...   
4              NaN           6.6           NaN  ...   
...            ...           ...           ...  ...   
6203           5.9           NaN           NaN  ...   
6204           5.6           NaN           NaN  ...   
6205           7.2           NaN           NaN  ...   
6206           7.1           NaN           NaN  ...   
6207           5.0           NaN           NaN  ...   

      Total Effects : Missing Description  Total Effects : Injuries  \
0                                     NaN                       NaN   
1                                     NaN                       NaN   
2                                     NaN                       4.0   
3                                     NaN                       NaN   
4                                     NaN                       2.0   
...                                   ...                       ...   
6203                                  NaN                     580.0   
6204                                  NaN                       NaN   
6205                                  NaN                       NaN   
6206                                  NaN                       3.0   
6207                                  NaN                       4.0   

      Total Effects : Injuries Description  \
0                                      NaN   
1                                      NaN   
2                    Few (~1 to 50 deaths)   
3                                      NaN   
4                    Few (~1 to 50 deaths)   
...                                    ...   
6203            Many (~101 to 1000 deaths)   
6204                                   NaN   
6205                                   NaN   
6206                 Few (~1 to 50 deaths)   
6207                 Few (~1 to 50 deaths)   

      Total Effects : Damages in million Dollars  \
0                                            NaN   
1                                            NaN   
2                                            NaN   
3                                            NaN   
4                                            0.4   
...                                          ...   
6203                                         NaN   
6204                                         NaN   
6205                                         NaN   
6206                                         NaN   
6207                                         NaN   

                     Total Effects : Damage Description  \
0                                                   NaN   
1                                                   NaN   
2     LIMITED (roughly corresponding to less than $1...   
3     LIMITED (roughly corresponding to less than $1...   
4     LIMITED (roughly corresponding to less than $1...   

In [20]:
db.columns

Index(['ID Earthquake', 'Flag Tsunami', 'Year', 'Month', 'Day', 'Focal Depth',
       'EQ Primary', 'Mw Magnitude', 'Ms Magnitude', 'Mb Magnitude',
       'Ml Magnitude', 'MFA Magnitude', 'Unknown Magnitude', 'Intensity',
       'Country', 'State', 'Location name', 'Region code',
       'Earthquake : Deaths', 'Earthquake : Deaths Description',
       'Earthquake : Missing', 'Earthquake : Missing Description',
       'Earthquake : Injuries', 'Earthquake : Injuries Description',
       'Earthquake : Damage (in M$)', 'Earthquake : Damage Description',
       'Earthquakes : Houses destroyed',
       'Earthquakes : Houses destroyed Description',
       'Earthquakes : Houses damaged',
       'Earthquakes : Houses damaged Description', 'Total Effects : Deaths',
       'Total Effects : Deaths Description', 'Total Effects : Missing',
       'Total Effects : Missing Description', 'Total Effects : Injuries',
       'Total Effects : Injuries Description',
       'Total Effects : Damages in million

In [21]:
db['Country'].values

array(['MEXICO', 'IRAN', 'ITALY', ..., 'PAPUA NEW GUINEA', 'INDONESIA',
       'CHINA'], dtype=object)

In [22]:
db = db.loc[(db['Country']=='CHILE')| (db['Country']=='USA') | (db['Country']=='JAPAN')]

In [23]:
db=db.drop(['State', 'Region code', 'Earthquake : Missing', 'Earthquake : Missing Description',
       'Earthquake : Injuries', 'Earthquake : Injuries Description', 'Total Effects : Deaths Description', 'Total Effects : Missing',
       'Total Effects : Missing Description', 'Total Effects : Injuries',
       'Total Effects : Injuries Description',
       'Total Effects : Damages in million Dollars',
       'Total Effects : Damage Description',
       'Total Effects : Houses Destroyed',
       'Total Effects : Houses Destroyed Description',
       'Total Effects : Houses Damaged',
       'Total Effects : Houses Damaged Description', 'Total Effects : Deaths'], axis=1)



In [24]:
db.columns

Index(['ID Earthquake', 'Flag Tsunami', 'Year', 'Month', 'Day', 'Focal Depth',
       'EQ Primary', 'Mw Magnitude', 'Ms Magnitude', 'Mb Magnitude',
       'Ml Magnitude', 'MFA Magnitude', 'Unknown Magnitude', 'Intensity',
       'Country', 'Location name', 'Earthquake : Deaths',
       'Earthquake : Deaths Description', 'Earthquake : Damage (in M$)',
       'Earthquake : Damage Description', 'Earthquakes : Houses destroyed',
       'Earthquakes : Houses destroyed Description',
       'Earthquakes : Houses damaged',
       'Earthquakes : Houses damaged Description', 'Coordinates'],
      dtype='object')

In [25]:
# Percentage of NaN values per column:

db.isna().sum()*100/len(db)

ID Earthquake                                  0.000000
Flag Tsunami                                  36.050517
Year                                           0.000000
Month                                          0.918485
Day                                            2.066590
Focal Depth                                   43.398393
EQ Primary                                    19.288175
Mw Magnitude                                  74.741676
Ms Magnitude                                  40.298507
Mb Magnitude                                  70.034443
Ml Magnitude                                  94.144661
MFA Magnitude                                 99.081515
Unknown Magnitude                             88.633754
Intensity                                     64.753157
Country                                        0.000000
Location name                                  0.000000
Earthquake : Deaths                           77.037887
Earthquake : Deaths Description               71

In [26]:
# Create a column with appropiate date data

db['Date']=pd.to_datetime(db[["Year", "Month", "Day"]],  errors = 'coerce')

In [27]:
# Separate the Coordinates values into Latitude and Longitude

lat = []
lon = []

for row in db['Coordinates']:
    try:
        lat.append(row.split(',')[0])
        lon.append(row.split(',')[1])
    except:
        lat.append(np.NaN)
        lon.append(np.NaN)

db['Latitude'] = lat
db['Longitude'] = lon

In [28]:
db

ID Earthquake Flag Tsunami  Year  Month   Day  Focal Depth  EQ Primary  \
6              6440      Tsunami  1972    6.0   8.0         39.0         6.6   
13             4878          NaN  1980    5.0  25.0          7.0         6.1   
15             4919          NaN  1981    4.0  26.0          6.0         6.0   
24             5203      Tsunami  1989    6.0  26.0          9.0         6.1   
33             5348      Tsunami  1993    2.0   7.0         11.0         6.3   
...             ...          ...   ...    ...   ...          ...         ...   
6184           7823          NaN  2008    4.0  18.0         14.0         5.3   
6185           8130          NaN  2008    5.0   7.0         39.0         6.8   
6192           9806          NaN  2011    3.0  11.0         19.0         7.6   
6201          10156      Tsunami  2015    9.0  16.0         22.0         8.3   
6202          10163      Tsunami  2015   11.0  13.0         12.0         6.7   

      Mw Magnitude  Ms Magnitude  Mb Magnitude  ...  \
6              NaN           6.6           6.2  ...   
13             NaN           6.1           6.1  ...   
15             NaN           6.0           5.5  ...   
24             NaN           6.1           5.8  ...   
33             6.3           6.2           6.3  ...   
...            ...           ...           ...  ...   
6184           5.3           4.8           5.1  ...   
6185           6.8           6.7           6.1  ...   
6192           7.6           NaN           7.1  ...   
6201           8.3           NaN           NaN  ...   
6202           6.7           NaN           NaN  ...   

      Earthquake : Damage (in M$)  \
6                             NaN   
13                            2.0   
15                            1.5   
24                            NaN   
33                            NaN   
...                           ...   
6184                          NaN   
6185                          NaN   
6192                          NaN   
6201                        600.0   
6202                          NaN   

                        Earthquake : Damage Description  \
6                                                   NaN   
13                         MODERATE (~$1 to $5 million)   
15                         MODERATE (~$1 to $5 million)   
24                         MODERATE (~$1 to $5 million)   
33    LIMITED (roughly corresponding to less than $1...   
...                                                 ...   
6184  LIMITED (roughly corresponding to less than $1...   
6185  LIMITED (roughly corresponding to less than $1...   
6192                                                NaN   
6201                     EXTREME (~$25 million or more)   
6202                                                NaN   

      Earthquakes : Houses destroyed  \
6                                NaN   
13                               NaN   
15                               NaN   
24                             105.0   
33                               NaN   
...                              ...   
6184                             NaN   
6185                             NaN   
6192                             NaN   
6201                          2305.0   
6202                             NaN   

      Earthquakes : Houses destroyed Description Earthquakes : Houses damaged  \
6                                            NaN                          NaN   
13                                           NaN                          NaN   
15                                           NaN                          NaN   
24                    Many (~101 to 1000 houses)                          NaN   
33                                           NaN                          NaN   
...                                          ...                          ...   
6184                                         NaN                          NaN   
6185                                         NaN                          NaN   
6192                

In [29]:
# drop columns Coordinates, Day, Month and Year

db = db.drop(['Coordinates', 'Day', 'Month', 'Year'], axis=1)


In [30]:
# Rename columns:

db.rename(columns={'ID Earthquake' : 'ID',
                   'Flag Tsunami' : 'Tsunami',
                   'EQ Primary' : 'Primary Magnitude',
                   'Location name' : 'Location',
                   'Earthquake : Deaths' : 'Deaths',
                   'Earthquake : Deaths Description' : 'Death Description',
                   'Earthquake : Damage (in M$)' : 'Damage (in M$)',
                   'Earthquake : Damage Description' : 'Damage Description',
                   'Earthquakes : Houses destroyed' : 'Houses Destroyed',
                   'Earthquakes : Houses destroyed Description' : 'Houses Destroyed Description',
                   'Earthquakes : Houses damaged' : 'Houses Damaged',
                   'Earthquakes : Houses damaged Description' : 'Houses Damaged Description'
                    }, inplace=True)


In [31]:
db.columns

Index(['ID', 'Tsunami', 'Focal Depth', 'Primary Magnitude', 'Mw Magnitude',
       'Ms Magnitude', 'Mb Magnitude', 'Ml Magnitude', 'MFA Magnitude',
       'Unknown Magnitude', 'Intensity', 'Country', 'Location', 'Deaths',
       'Death Description', 'Damage (in M$)', 'Damage Description',
       'Houses Destroyed', 'Houses Destroyed Description', 'Houses Damaged',
       'Houses Damaged Description', 'Date', 'Latitude', 'Longitude'],
      dtype='object')

In [32]:
# Rearrange columns order:

db = db [['ID', 'Date', 'Country', 'Location', 'Latitude', 'Longitude', 'Focal Depth', 'Primary Magnitude',
       'Mw Magnitude', 'Ms Magnitude', 'Mb Magnitude', 'Ml Magnitude',
       'MFA Magnitude', 'Unknown Magnitude', 'Intensity', 'Deaths', 'Death Description', 'Damage (in M$)',
       'Damage Description', 'Houses Destroyed',
       'Houses Destroyed Description', 'Houses Damaged',
       'Houses Damaged Description', 'Tsunami']]

In [33]:
# Replace Tsunami values with 1 or 0:

db['Tsunami'] = db['Tsunami'].replace('Tsunami', 1)

db['Tsunami'] = db['Tsunami'].replace(np.nan, 0)

In [34]:
# Clasification of Depth and Magnitude

db['Class_Depth'] = pd.cut(
    x=db["Focal Depth"],
    bins=[0, 49, 300, np.inf],
    labels=["Shallow", "Intermediate", "Deep"],
)

db['Class_Mag'] = pd.cut(
    x=db["Primary Magnitude"],
    bins=[0, 2.9, 3.9,4.9,5.9,6.9,7.9, np.inf],
    labels=["Very Minor", "Minor", "Light","Moderate","Strong","Major","Great"],
)

In [35]:
db = db [['ID', 'Date', 'Country', 'Location', 'Latitude', 'Longitude', 'Focal Depth', 'Class_Depth', 'Primary Magnitude', 'Class_Mag',
       'Mw Magnitude', 'Ms Magnitude', 'Mb Magnitude', 'Ml Magnitude',
       'MFA Magnitude', 'Unknown Magnitude', 'Intensity', 'Deaths', 'Death Description', 'Damage (in M$)',
       'Damage Description', 'Houses Destroyed',
       'Houses Destroyed Description', 'Houses Damaged',
       'Houses Damaged Description', 'Tsunami']]

In [36]:
#enrich the Newest Depth Column with real data found in internet articles
db.loc[db['ID']==10036,['Focal Depth']]=2
db.loc[db['ID']==7614,['Focal Depth']]=5
db.loc[db['ID']==5538,['Focal Depth']]=20
db.loc[db['ID']==9813,['Focal Depth']]=25
db.loc[db['ID']==4248,['Focal Depth']]=20

With an analysis we determine that the Ms Magnitude and the Mw Magnitude have a diference of just 0.15 on avg in this dataset.
So we kept the Primary magnitude that is based on those magnitudes taking into account that the Mw magnitude is the most accurate, the primary magnitude that are based on the other magnitudes are transform into 0.

In [37]:
db['Primary Magnitude'] = np.where((db['Primary Magnitude'] == db['Ms Magnitude']) | (db['Primary Magnitude'] == db['Mw Magnitude']), db['Primary Magnitude'],0)

In [38]:
#Drop the primary magnitudes that are 0.
db.drop(db[db['Primary Magnitude'] == 0].index, inplace = True)

In [39]:
#Drop the columns of magnitudes that are not primary
db.drop(['Mw Magnitude','Ms Magnitude','Mb Magnitude', 'Ml Magnitude','MFA Magnitude','Unknown Magnitude'], axis=1,inplace=True)

In [40]:
#Check the database
db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 594 entries, 6 to 6202
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   ID                            594 non-null    int64         
 1   Date                          528 non-null    datetime64[ns]
 2   Country                       594 non-null    object        
 3   Location                      594 non-null    object        
 4   Latitude                      594 non-null    object        
 5   Longitude                     594 non-null    object        
 6   Focal Depth                   404 non-null    float64       
 7   Class_Depth                   399 non-null    category      
 8   Primary Magnitude             594 non-null    float64       
 9   Class_Mag                     594 non-null    category      
 10  Intensity                     229 non-null    float64       
 11  Deaths                        1

Determine the avg values of Deaths according to the Death Description. 
So we can have a more complete Deaths column.

In [41]:
db.groupby('Death Description', as_index=False)['Deaths'].mean()

Death Description        Deaths
0             Few (~1 to 50 deaths)      9.201923
1        Many (~101 to 1000 deaths)    430.555556
2          Some (~51 to 100 deaths)     69.500000
3  Very Many (~1001 or more deaths)  15090.761905

In [42]:
#Fill the Na on Deaths column with 0.
db['Deaths'] = db['Deaths'].fillna(0)

In [43]:
#Change the type of the column Death Description to a string.
db['Death Description']=db['Death Description'].astype(str)

In [44]:
#Check the Death Description column 
db['Death Description'].head(30)

6                        nan
13                       nan
15                       nan
24                       nan
33                       nan
35                       nan
49                       nan
51                       nan
57                       nan
64                       nan
71                       nan
78                       nan
94                       nan
97                       nan
115                      nan
131                      nan
138                      nan
147    Few (~1 to 50 deaths)
160                      nan
170                      nan
184                      nan
192                      nan
204                      nan
205                      nan
218                      nan
221                      nan
239                      nan
244                      nan
260                      nan
265                      nan
Name: Death Description, dtype: object

In [45]:
#Create a column tmp that would have the previously search avg, according to the Death description column.
db['tmp_table']=''
conditions=[(db['Death Description'].str.contains('Few')),(db['Death Description'].str.contains('101')),(db['Death Description'].str.contains('Some')),(db['Death Description'].str.contains('more'))]
choices=[9,430,70,15090]
db['tmp_table']=np.select(conditions,choices)

In [46]:
db['tmp_table'].head(30)

6      0
13     0
15     0
24     0
33     0
35     0
49     0
51     0
57     0
64     0
71     0
78     0
94     0
97     0
115    0
131    0
138    0
147    9
160    0
170    0
184    0
192    0
204    0
205    0
218    0
221    0
239    0
244    0
260    0
265    0
Name: tmp_table, dtype: int64

In [47]:
#Insert the values of tmp column where the Death column is 0.
db['Deaths'] = np.where((db['Deaths'] == 0), db['tmp_table'],db['Deaths'])

In [48]:
#delete the already use tmp column
del db['tmp_table']

We will reapeated the process with the Damage columns, Houses Destroyed colums and Houses Damage Column.

In [49]:
db['Deaths'].head(25)

6      0.0
13     0.0
15     0.0
24     0.0
33     0.0
35     0.0
49     0.0
51     0.0
57     0.0
64     0.0
71     0.0
78     0.0
94     0.0
97     0.0
115    0.0
131    0.0
138    0.0
147    9.0
160    0.0
170    0.0
184    0.0
192    0.0
204    0.0
205    0.0
218    0.0
Name: Deaths, dtype: float64

In [50]:
db.groupby('Damage Description', as_index=False)['Damage (in M$)'].mean()

Damage Description  Damage (in M$)
0                     EXTREME (~$25 million or more)     4913.343800
1  LIMITED (roughly corresponding to less than $1...        0.486000
2                       MODERATE (~$1 to $5 million)        2.195714
3                       SEVERE (~>$5 to $24 million)       11.560000

In [51]:
db['Damage (in M$)'] = db['Damage (in M$)'].fillna(0)

In [52]:
db['Damage Description']=db['Damage Description'].astype(str)

In [53]:
db['tmp_table']=''
conditions=[(db['Damage Description'].str.contains('LIMITED')),(db['Damage Description'].str.contains('MODERATE')),(db['Damage Description'].str.contains('SEVERE')),(db['Damage Description'].str.contains('EXTREME'))]
choices=[0.5,2,11,4900]
db['tmp_table']=np.select(conditions,choices)

In [54]:
db['Damage (in M$)'] = np.where((db['Damage (in M$)'] == 0), db['tmp_table'],db['Damage (in M$)'])

In [55]:
del db['tmp_table']

In [56]:
db.groupby('Houses Destroyed Description', as_index=False)['Houses Destroyed'].mean()

Houses Destroyed Description  Houses Destroyed
0             Few (~1 to 50 houses)         17.600000
1        Many (~101 to 1000 houses)        401.615385
2          Some (~51 to 100 houses)         69.888889
3  Very Many (~1001 or more houses)      27405.625000

In [57]:
db['Houses Destroyed'] = db['Houses Destroyed'].fillna(0)

In [58]:
db['Houses Destroyed Description']=db['Houses Destroyed Description'].astype(str)

In [59]:
db['tmp_table']=''
conditions=[(db['Houses Destroyed Description'].str.contains('Few')),(db['Houses Destroyed Description'].str.contains('101')),(db['Houses Destroyed Description'].str.contains('Some')),(db['Houses Destroyed Description'].str.contains('more'))]
choices=[18,402,70,27406]
db['tmp_table']=np.select(conditions,choices)

In [60]:
db['Houses Destroyed'] = np.where((db['Houses Destroyed'] == 0), db['tmp_table'],db['Houses Destroyed'])

In [61]:
del db['tmp_table']

In [62]:
db.groupby('Houses Damaged Description', as_index=False)['Houses Damaged'].mean()

Houses Damaged Description  Houses Damaged
0             Few (~1 to 50 houses)       38.400000
1        Many (~101 to 1000 houses)      463.000000
2          Some (~51 to 100 houses)       78.666667
3  Very Many (~1001 or more houses)    38640.266667

In [63]:
db['Houses Damaged'] = db['Houses Damaged'].fillna(0)

In [64]:
db['Houses Damaged Description']=db['Houses Damaged Description'].astype(str)

In [65]:
db['tmp_table']=''
conditions=[(db['Houses Damaged Description'].str.contains('Few')),(db['Houses Damaged Description'].str.contains('101')),(db['Houses Damaged Description'].str.contains('Some')),(db['Houses Damaged Description'].str.contains('more'))]
choices=[38,463,79,38640]
db['tmp_table']=np.select(conditions,choices)

In [66]:
db['Houses Damaged'] = np.where((db['Houses Damaged'] == 0), db['tmp_table'],db['Houses Damaged'])

In [67]:
del db['tmp_table']

In [68]:
# Final view of finished dataset:

db

ID       Date Country                                    Location  \
6      6440 1972-06-08   CHILE                             CHILE:  CENTRAL   
13     4878 1980-05-25     USA                  CALIFORNIA:  MAMMOTH LAKES   
15     4919 1981-04-26     USA         CALIFORNIA:  WESTMORLAND,CALIPATRIA   
24     5203 1989-06-26     USA            HAWAIIAN ISLANDS:  PUNA DISTRICT   
33     5348 1993-02-07   JAPAN  JAPAN:  HONSHU:  ISHIKAWA, TOYAMA, NIIGATA   
...     ...        ...     ...                                         ...   
6184   7823 2008-04-18     USA                       ILLINOIS:  WEST SALEM   
6185   8130 2008-05-07   JAPAN                    JAPAN:  HONSHU:  E COAST   
6192   9806 2011-03-11   JAPAN               JAPAN:  OFF EAST COAST HONSHU   
6201  10156 2015-09-16   CHILE                             CHILE:  CENTRAL   
6202  10163 2015-11-13   JAPAN                       JAPAN:  KYUSYU ISLAND   

     Latitude Longitude  Focal Depth Class_Depth  Primary Magnitude Class_Mag  \
6       -30.5     -71.8         39.0     Shallow                6.6    Strong   
13       37.6   -118.84          7.0     Shallow                6.1    Strong   
15     33.133   -115.65          6.0     Shallow                6.0    Strong   
24     19.362  -155.083          9.0     Shallow                6.1    Strong   
33     37.634   137.245         11.0     Shallow                6.3    Strong   
...       ...       ...          ...         ...                ...       ...   
6184   38.452   -87.886         14.0     Shallow                5.3  Moderate   
6185   36.158   141.521         39.0     Shallow                6.8    Strong   
6192   38.058    144.59         19.0     Shallow                7.6     Major   
6201  -31.573   -71.674         22.0     Shallow                8.3     Great   
6202   31.001   128.873         12.0     Shallow                6.7    Strong   

      Intensity  Deaths      Death Description  Damage (in M$)  \
6           7.0     0.0                    nan             0.0   
13          7.0     0.0                    nan             2.0   
15          7.0     0.0                    nan             1.5   
24          6.0     0.0                    nan             2.0   
33          NaN     0.0                    nan             0.5   
...         ...     ...                    ...             ...   
6184        NaN     0.0                    nan             0.5   
6185        NaN     0.0                    nan             0.5   
6192        NaN     0.0                    nan             0.0   
6201        9.0     7.0  Few (~1 to 50 deaths)           600.0   
6202        4.0     0.0                    nan             0.0   

                                     Damage Description  Houses Destroyed  \
6                                                   nan               0.0   
13                         MODERATE (~$1 to $5 million)               0.0   
15                         MODERATE (~$1 to $5 million)               0.0   
24                         MODERATE (~$1 to $5 million)             105.0   
33    LIMITED (roughly corresponding to less than $1...               0.0   
...                                                 ...               ...   
6184  LIMITED (roughly corresponding to less than $1...               0.0   
6185  LIMITED (roughly corresponding to less than $1...               0.0   
6192                                                nan               0.0   
6201                     EXTREME (~$25 million or more)            2305.0   
6202                                                nan               0.0   

          Houses Destroyed Description  Houses Damaged  \
6                                  nan             0.0   
13                                 nan             0.0   
15                                 nan             0.0   
24          Many (~101 to 1000 houses)             0.0   
33                                 nan             0.0   
...                              

In [70]:
db.to_csv ('Datasets\Output\Significant-Earthquake-Database.csv')